# 데이터 불러오기
* Distance의 경우 공항간의 직선 거리를 활용하였음

In [1]:
from google.colab import drive
drive.mount('/gdrive')
import pandas as pd
SFSNT = pd.read_csv('/gdrive/My Drive/빅콘테스트/빅콘테스트_수정본/SFSNT.csv') # 2019년 하계 스케줄 중 7월~9월이 포함된 시즌데이터
AFSNT = pd.read_csv('/gdrive/My Drive/빅콘테스트/빅콘테스트_수정본/AFSNT.csv') # 2017.1.1 ~ 2019. 6.30 까지의 운항실적 데이터(학습데이터)
AFSNT_DLY = pd.read_csv('/gdrive/My Drive/빅콘테스트/빅콘테스트_수정본/AFSNT_DLY.csv') # 2019.9.16 ~ 2019.9.30 까지 지연율 계산
Distance = pd.read_csv('/gdrive/My Drive/빅콘테스트/공항별 거리1.csv',header=None)
gonghu = pd.read_csv('/gdrive/My Drive/빅콘테스트/공휴일.csv')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [2]:
AFSNT_DLY.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,AOD,STT,DLY,DLY_RATE
0,2019,9,16,월,ARP1,ARP3,L,L1702,A,9:05,NaN,NaN
1,2019,9,16,월,ARP3,ARP1,L,L1702,D,7:55,NaN,NaN
2,2019,9,16,월,ARP1,ARP3,L,L1720,A,14:40,NaN,NaN
3,2019,9,16,월,ARP3,ARP1,L,L1720,D,13:30,NaN,NaN
4,2019,9,16,월,ARP4,ARP3,L,L1808,A,20:10,NaN,NaN


# 공휴일 feature를 추가하기 위한 과정

In [3]:
gonghu.values
#head()

array([['2017-01-01', 1.0],
       ['2017-01-02', nan],
       ['2017-01-03', nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=object)

In [0]:
gonghu = gonghu.dropna()

In [0]:
gonghu = gonghu["날짜"].values

In [0]:
d = []
for i in range(len(gonghu)):
    a,b,c = int(gonghu[i][0:4]),int(gonghu[i][5:7]), int(gonghu[i][8:])
    d.append([a,b,c])

In [0]:
dic = {}
for i in range(len(d)):
    dic[tuple(d[i])] = 'Y'

In [0]:
date = AFSNT[['SDT_YY', 'SDT_MM', 'SDT_DD']].values

In [0]:
k = []
for i in range(len(date)):
    try:
        k.append(dic[tuple(date[i])])
    except:
        k.append('N')

In [0]:
AFSNT["공휴일"] = k

# 시간을 수치화 하기 위한 전처리 과정
* [(시간 * 60) + 분]을 활용하여 모든 시간을 수치화 시킴
* 시간만을 판단할 경우 전처리 과정은 내림 하였음

In [0]:
# 시간 전처리
STT = AFSNT["STT"].values
ATT = AFSNT["ATT"].values

STT_time = []
STT_minute = []
ATT_time = []
ATT_minute = []
for i in range(len(STT)):
    if STT[i][1] == ":":
        STT_time.append(int(STT[i][0]))
    else:
        STT_time.append(int(STT[i][0] + STT[i][1]))
for i in range(len(STT)):
    if STT[i][1] == ":":
        STT_minute.append(int(STT[i][2] + STT[i][3]))
    else:
        STT_minute.append(int(STT[i][3] + STT[i][4]))

for i in range(len(ATT)):
    if ATT[i][1] == ":":
        ATT_time.append(int(ATT[i][0]))
    else:
        ATT_time.append(int(ATT[i][0] + ATT[i][1]))
for i in range(len(ATT)):
    if ATT[i][1] == ":":
        ATT_minute.append(int(ATT[i][2] + ATT[i][3]))
    else:
        ATT_minute.append(int(ATT[i][3] + ATT[i][4]))
        
STT_total = []
ATT_total = []
for i in range(len(STT_time)):
    STT_total.append(STT_time[i] * 60 + STT_minute[i])
    ATT_total.append(ATT_time[i] * 60 + ATT_minute[i])
late_number = []
for i in range(len(STT_total)):
    if ATT_total[i] - STT_total[i] > 30:
        late_number.append(i)
        
# ((60 * 시간) + )분 이용, 시간을 수치화
AFSNT["STT_total"] = STT_total
AFSNT["ATT_total"] = ATT_total
AFSNT["STT_time"] = STT_time
AFSNT["ATT_time"] = ATT_time

In [12]:
AFSNT.head()

,SDT_YY,SDT_MM,SDT_DD,SDT_DY,ARP,ODP,FLO,FLT,REG,AOD,IRR,STT,ATT,DLY,DRR,CNL,CNR,공휴일,STT_total,ATT_total,STT_time,ATT_time
0,2017,1,1,일,ARP3,ARP6,J,J1955,SEw3NzE4,D,N,10:05,10:32,N,NaN,N,NaN,Y,605,632,10,10
1,2017,1,1,일,ARP3,ARP6,J,J1954,SEw3NzE4,A,N,9:30,9:31,N,NaN,N,NaN,Y,570,571,9,9
2,2017,1,1,일,ARP3,ARP6,J,J1956,SEw3NzE4,A,N,12:45,13:03,N,NaN,N,NaN,Y,765,783,12,13
3,2017,1,1,일,ARP3,ARP6,J,J1957,SEw3NzE4,D,N,13:25,14:09,Y,C02,N,NaN,Y,805,849,13,14
4,2017,1,1,일,ARP3,ARP6,J,J1958,SEw3NzE4,A,N,16:10,16:31,N,NaN,N,NaN,Y,970,991,16,16


In [13]:
AFSNT[AFSNT["CNL"].isin(['Y'])]["CNR"].values

array(['A02', 'A02', 'A02', ..., 'A05', 'A05', 'A05'], dtype=object)

# 공항간 거리 feature를 추가하는 과정

In [14]:
Distance

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,328,451,268,317,111,287,240,318,172,289,295,180,104,33
2,0,328,0,291,190,59,215,234,85,126,321,102,79,227,268,339
3,0,451,291,0,183,349,368,166,330,180,542,384,229,268,459,439
4,0,268,190,183,0,236,186,42,185,80,364,255,116,89,278,260
5,0,317,59,349,236,0,208,277,73,180,280,43,128,250,241,332
6,0,111,215,368,186,208,0,216,137,209,181,191,188,118,92,124
7,0,287,234,166,42,277,216,0,228,113,397,297,155,106,309,275
8,0,240,85,330,185,73,137,228,0,150,240,72,102,182,183,261
9,0,318,126,180,80,180,209,113,150,0,370,208,51,150,293,309


In [15]:
ARP_to_int = {'ARP1':1,
'ARP2':2,
'ARP3':3,
'ARP4':4,
'ARP5':5,
'ARP6':6,
'ARP7':7,
'ARP8':8,
'ARP9':9,
'ARP10':10,
'ARP11':11,
'ARP12':12,
'ARP13':13,
'ARP14':14,
'ARP15':15}

AFSNT["ODP"] = AFSNT["ODP"].apply(lambda x: ARP_to_int[x])
AFSNT["ARP"] = AFSNT["ARP"].apply(lambda x: ARP_to_int[x])

AFSNT_DLY["ODP"] = AFSNT_DLY["ODP"].apply(lambda x: ARP_to_int[x])
AFSNT_DLY["ARP"] = AFSNT_DLY["ARP"].apply(lambda x: ARP_to_int[x])

ARP = AFSNT["ARP"].values
ODP = AFSNT["ODP"].values

ARP_EX = AFSNT_DLY["ARP"].values
ODP_EX = AFSNT_DLY["ODP"].values

new = []
for i in range(len(ARP)):
    a, b = ARP[i], ODP[i]
    d = Distance[a][b]
    new.append(d)
AFSNT["Distance"] = new

new_EX = []
for i in range(len(ARP_EX)):
    a, b = ARP[i], ODP[i]
    d = Distance[a][b]
    new_EX.append(d)
AFSNT["Distance"] = new_EX

ValueError: ignored

# 목적 함수인 (STT - ATT)를 구하기 위한 과정

In [0]:
NEW = AFSNT["STT_total"].values - AFSNT["ATT_total"].values

In [0]:
SDT_YY = AFSNT["SDT_YY"].values
SDT_MM = AFSNT["SDT_MM"].values
SDT_DD = AFSNT["SDT_DD"].values

In [0]:
from datetime import datetime, timedelta
differ_time = []
for i in range(len(AFSNT)):
    time1 = datetime(SDT_YY[i], SDT_MM[i], SDT_DD[i], STT_time[i], STT_minute[i], 0)
    time2 = datetime(SDT_YY[i], SDT_MM[i], SDT_DD[i], ATT_time[i], ATT_minute[i], 0)
    differ = (time2 - time1).seconds / 60
    if differ > 1300:
        differ_time.append(differ-1440)
    else:
        differ_time.append(differ)
AFSNT["Differ"] = differ_time

In [0]:
AFSNT.head()

In [0]:
AFSNT = AFSNT.drop(["STT","ATT"],axis=1)

# 범주형 변수를 수치화 하기 위한 과정
* 사용 모델이 Decision Tree 계열이기 때문에 범주형 변수를 one-hot encoding과 같은 방식을 사용할 필요가 없다.

In [0]:
AFSNT_DLY.head()

In [0]:
try:
    AFSNT_DLY["FLT"] = AFSNT_DLY["FLT"].apply(lambda x: flt_map[x])
except:
    

In [0]:
flo_not = list(set(AFSNT_DLY["FLO"].values) - set(flo))
for i in range(len(flo_not)):
    flo_map[flo_not[i]] = 1541

In [0]:
flo_map

In [0]:
max(flt_map.values())

In [0]:
set(flt)

In [0]:
# 공항 전처리
AFSNT["ARP"] = AFSNT["ARP"].apply(lambda x: ARP_to_int[x])

# FLT 전처리
flt = AFSNT["FLT"].values
flt = set(flt)
flt = list(flt)
flt_map = {}
flt_not = list(set(AFSNT_DLY["FLT"].values) - set(flt))
for i in range(len(flt_not)):
    flt_map[flt_not[i]] = 1541
    
for i in range(len(flt)):
    flt_map[flt[i]] = i
AFSNT["FLT"] = AFSNT["FLT"].apply(lambda x: flt_map[x])
AFSNT_DLY["FLT"] = AFSNT_DLY["FLT"].apply(lambda x: flt_map[x])
# FLO 전처리
flo = AFSNT["FLO"].values
flo = set(flo)
flo = list(flo)
flo_map = {}
for i in range(len(flo)):
    flo_map[flo[i]] = i
AFSNT["FLO"] = AFSNT["FLO"].apply(lambda x: flo_map[x])
# REG 전처리
reg = AFSNT["REG"].values
reg = set(reg)
reg = list(reg)
reg_map = {}
for i in range(len(reg)):
    reg_map[reg[i]] = i
AFSNT["REG"] = AFSNT["REG"].apply(lambda x: reg_map[x])
# DRR 전처리
drr = AFSNT["DRR"].values
drr = set(drr)
drr = list(drr)
drr_map = {}
for i in range(len(drr)):
    drr_map[drr[i]] = i
AFSNT["DRR"] = AFSNT["DRR"].apply(lambda x: drr_map[x])
AFSNT["CNR"] = AFSNT["CNR"].apply(lambda x: drr_map[x])
# AOD, IRR, DLY, CNL,공휴일 전처리
AOD = {'A' : 1, 'D' : 0}
IRR = {'N' : 0, 'Y' : 1}
DLY = {'N' : 0, 'Y' : 1}
CNL = {'N' : 0, 'Y' : 1}
Holiday = {'Y':1, 'N':0}
AFSNT["AOD"] = AFSNT["AOD"].apply(lambda x: AOD[x])
AFSNT["IRR"] = AFSNT["IRR"].apply(lambda x: IRR[x])
AFSNT["DLY"] = AFSNT["DLY"].apply(lambda x: DLY[x])
AFSNT["CNL"] = AFSNT["CNL"].apply(lambda x: CNL[x])
AFSNT["공휴일"] = AFSNT["공휴일"].apply(lambda x: Holiday[x])

# 날씨 정보를 요약하기 위한 과정
* 데이터를 항공 기상청에 있는 7개 공항의 데이터를 csv파일로 불러오고 나머지 8개 공항의 날씨는 기상청에 있는 데이터를 활용하였음
* 기상청 데이터를 활용 할 때는 가까운 지역으로 하였음
* 전처리에 있어서 항공 기상청의 7개과 기상청의 8개 사이에서 활용할 수 있는 feature들은 모두 활용하였다. 
* 습도(%),현지기압(hPa),해면기압(hPa),강수량(mm),이슬점온도(°C),기온(°C),시정(10m),풍속(m/s) 등으로 구성하였음
* 날씨를 정확하게 예측할 수 없기 때문에 시간별 지연 대수를 날씨 변수들과 매핑을 시켰다. 각 공항 별로 지연 대수의 분포를 활용하여 33% 간격으로 상 중 하로 나누어 답으로 놓고 딥러닝 모형을 만들었다.

In [0]:
Weather = pd.read_csv('/gdrive/My Drive/빅콘테스트/15개공항표준화/Weather.csv')

In [0]:
Weather = Weather[['일시', '습도(%)', '현지기압(hPa)', '해면기압(hPa)',
       '강수량(mm)', '이슬점온도(°C)', '기온(°C)', '시정(10m)', '풍속(m/s)_new', 'AOD_A',
       'AOD_D', 'ARP_ARP1', 'ARP_ARP10', 'ARP_ARP11', 'ARP_ARP12', 'ARP_ARP13',
       'ARP_ARP14', 'ARP_ARP15', 'ARP_ARP2', 'ARP_ARP3', 'ARP_ARP4',
       'ARP_ARP5', 'ARP_ARP6', 'ARP_ARP7', 'ARP_ARP8', 'ARP_ARP9',
       'number_High', 'number_Middle', 'number_low']]

In [0]:
Weather_train = Weather[['일시', '습도(%)', '현지기압(hPa)', '해면기압(hPa)',
       '강수량(mm)', '이슬점온도(°C)', '기온(°C)', '시정(10m)', '풍속(m/s)_new', 'AOD_A',
       'AOD_D', 'ARP_ARP1', 'ARP_ARP10', 'ARP_ARP11', 'ARP_ARP12', 'ARP_ARP13',
       'ARP_ARP14', 'ARP_ARP15', 'ARP_ARP2', 'ARP_ARP3', 'ARP_ARP4',
       'ARP_ARP5', 'ARP_ARP6', 'ARP_ARP7', 'ARP_ARP8', 'ARP_ARP9']]
Weather_answer = Weather[['number_High', 'number_Middle', 'number_low']]

In [0]:
from sklearn.utils import shuffle
Weather_train_suffle , Weather_answer_suffle = shuffle(Weather_train, Weather_ans)

In [0]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
import tensorflow as tf

# parameters
learning_rate = 0.001
training_epochs = 25
batch_size = 30

# input place holders
X = tf.placeholder(tf.float32, [None, 26])
Y = tf.placeholder(tf.float32, [None, 3])

# weights & bias for nn layers
W1 = tf.get_variable("ARP_W1", shape=[26, 20], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([20]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.get_variable("ARP_W2", shape=[20, 40], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([40]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)

W3 = tf.get_variable("ARP_W3", shape=[40, 40], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([40]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)

W4 = tf.get_variable("ARP_W4", shape=[40, 20], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([20]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)

W5 = tf.get_variable("ARP_W5", shape=[20, 3], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([3]))
hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=hypothesis, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initialize
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# train my model
for epoch in range(training_epochs):
    avg_cost = 0
    total_batch = int(ARP1.shape[0] / batch_size)

    for i in range(total_batch): 
        batch_xs, batch_ys = Weather_train_suffle[i*batch_size:(i+1)*batch_size], Weather_answer_suffle[i*batch_size:(i+1)*batch_size]
        feed_dict = {X : batch_xs, Y : batch_ys}
        c, _ = sess.run([cost, optimizer], feed_dict=feed_dict)
        avg_cost += c / total_batch

    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.9f}'.format(avg_cost))

print('Learning Finished!')

# Test model and check accuracy
correct_prediction = tf.equal(tf.argmax(hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print('Accuracy:', sess.run(accuracy, feed_dict={
      X: Weather_train, Y: Weather_ans}))

In [0]:
AFSNT_weather = AFSNT_weather = AFSNT_weather[["0", "('습도(%)',)", "('현지기압(hPa)',)", "('해면기압(hPa)',)",
       "('강수량(mm)',)", "('이슬점온도(°C)',)", "('기온(°C)',)", "('시정(10m)',)",
       "('풍속(m/s)_new',)", "AOD_A", "AOD_D", "ARP_ARP1", "ARP_ARP10",
       "ARP_ARP11", "ARP_ARP12", "ARP_ARP13", "ARP_ARP14", "ARP_ARP15",
       "ARP_ARP2", "ARP_ARP3", "ARP_ARP4", "ARP_ARP5", "ARP_ARP6", "ARP_ARP7",
       "ARP_ARP8", "ARP_ARP9"]]

In [0]:
pred = []
for i in range(len(AFSNT_weather)):
    pred.append(sess.run(tf.argmax(hypothesis, 1), feed_dict={X:  AFSNT_weather.loc[[i]].values}))

In [0]:
AFSNT['Weather'] = pred

In [0]:
Weather.columns

In [0]:
AFSNT["Differ"] = differ_time

In [0]:
AFSNT.head()

In [0]:
AFSNT.drop(["STT_total","ATT_total"],axis=1)

In [0]:
AFSNT= differ_time

In [0]:
# airport = ['김포','김해','제주','광주','울산','청주','무안','대구','여수','양양','포항','사천','군산','원주','인천']

import numpy as np
cna = np.sort(NEW)

In [0]:
cna

In [0]:
NEW = list(NEW)
NEW.index(-765)

In [0]:
AFSNT[466329:].head()

In [0]:
AFSNT = pd.concat([AFSNT.drop('SDT_DY', axis=1), pd.get_dummies(AFSNT['SDT_DY'])], axis=1)

In [0]:
AFSNT = AFSNT.drop(['STT','ATT','STT_time','ATT_time'],axis = 1)

In [0]:
Differ = AFSNT["Differ"]

In [0]:
AFSNT = AFSNT.drop(['Differ'],axis = 1)

In [0]:
AFSNT

In [0]:
AFSNT.to_csv("/gdrive/My Drive/빅콘테스트/빅콘테스트_수정본/AFSNT_preprocessing_for decision tree.csv", mode='w')

In [0]:
Differ = AFSNT["Differ"]

In [0]:
AFSNT = AFSNT.drop(["Differ"],axis = 1)

In [0]:
AFSNT.head()

In [0]:
len(AFSNT)/4

In [0]:
x_train = AFSNT[:246927]
x_test = AFSNT[-246927:]
y_train = Differ[:246927]
y_test = Differ[-246927:]

In [0]:
x_test.head()

In [0]:
AFSNT

In [0]:
DY = {'MON':1,
'TUE':2,
'WED':3,
'THU':4,
'FRI':5,
'SAT':6,
'SUN':7}
AFSNT["SDT_DY"] = AFSNT["SDT_DY"].apply(lambda x: DY[x])

In [0]:
gonghu = {'Y':1,'N':0}
AFSNT["공휴일"] = AFSNT["공휴일"].apply(lambda x: gonghu[x])

In [0]:
gonghu = {'Y':1,'N':0}
AFSNT["공휴일"] = AFSNT["공휴일"].apply(lambda x: gonghu[x])

In [0]:
ARP = AFSNT["ARP"].values

In [0]:
ODP = AFSNT["ODP"].values

In [0]:
ARP[0]

In [0]:
for i in range(len(ARP)):
    ARP[i] = ARP[i][-1]

In [0]:
for i in range(len(ODP)):
    ODP[i] = ODP[i][-1]

In [0]:
AFSNT["ARP"] = ARP
AFSNT["ODP"] = ODP

In [0]:
ATT_total

In [0]:
AFSNT

In [0]:
AFSNT = AFSNT.drop(['ATT_total'],axis = 1)

In [0]:
AFSNT.head()

In [0]:
from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 0)
regressor.fit(AFSNT,Differ)

In [0]:
regressor

In [0]:
DY = {'월':1,
'화':2,
'수':3,
'목':4,
'금':5,
'토':6,
'일':7}
AFSNT_DLY["SDT_DY"] = AFSNT_DLY["SDT_DY"].apply(lambda x: DY[x])

In [0]:
AOD = {'A' : 1, 'D' : 0}
IRR = {'N' : 0, 'Y' : 1}
DLY = {'N' : 0, 'Y' : 1}
CNL = {'N' : 0, 'Y' : 1}
AFSNT["AOD"] = AFSNT["AOD"].apply(lambda x: AOD[x])
#AFSNT["IRR"] = AFSNT["IRR"].apply(lambda x: IRR[x])
AFSNT["DLY"] = AFSNT["DLY"].apply(lambda x: DLY[x])
#AFSNT["CNL"] = AFSNT["CNL"].apply(lambda x: CNL[x])

In [0]:
AFSNT = AFSNT.drop(['DRR','CNL','CNR'],axis = 1)
#AFSNT[['DRR','CNL','CNR']]

In [0]:
AFSNT.head()

In [0]:
AFSNT_DLY.head()

In [0]:
regr_2 = DecisionTreeRegressor(max_depth=10)
regr_2.fit(x_train,y_train)

In [0]:
model = tree.DecisionTreeClassifier(criterion'gini') # for classification, here you can change the algorithm as gini or entropy (information gain) by default it is gini
# model = tree.DecisionTreeRegressor() for regression
# Train the model using the training sets and check score
model.fit(X, y)
model.score(X, y)
#Predict Output
predicted = model.predict(x_test)

In [0]:
# Import the necessary modules and libraries
import numpy as np
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=10)
regr_1.fit(x_train,y_train)
regr_2.fit(x_train,y_train)

# Predict
y_1 = regr_1.predict(x_test)
y_2 = regr_2.predict(x_test)

from sklearn.metrics import mean_squared_error 
mean_squared_error(y_test, y_1)
mean_squared_error(y_test, y_2)

In [0]:
import matplotlib.pyplot as plt